# Testing the MLE Fitting Functions

The Hawkes module implements the multivariate Hawkes marked point process. We assume an exponential decay function is used to improve the numerical computation of the intensity function [Liniger, 2012].

Generating artificial points to be used as data for fit (see simulation notebook):

In [106]:
import hawkes
import markDistributions as md
import numpy as np
import scipy.optimize as op
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from pylab import rcParams

%matplotlib inline
np.set_printoptions(precision=3, suppress=True)
rcParams['figure.figsize'] = 16, 5

d=2
numPoints=7200

eta=[0.021,0.029]
Q=[0.61,0.16,0.6,0.06]
alpha=[0.005]*d
mu=[3.6,4.2]
rho=[5.6,7.2]
alph=[0.47,1.1]
beta=[0.22,0.0]
gmma=[0.0,0.0]

mdParam1=[alph[0],beta[0],gmma[0]]
mdParam2=[alph[1],beta[1],gmma[1]]

sparam = []
sparam.extend(eta)
sparam.extend(Q)
sparam.extend(alpha)
sparam.extend(mdParam1)
sparam.extend(mdParam2)

markparam = [[mu[0],rho[0]],[mu[1],rho[1]]]

sim = hawkes.Hawkes(d=d)
sim.setParam(sparam)
sim.setMarkDistParam(markparam)

t,j,x = sim.simulate(numPoints)[0]

Setup a seperate Hawkes marked point process:

In [107]:
h = hawkes.Hawkes(d=d)
h.setMarkDistParam(markparam)

Fit the new Hawkes process to the simulated data using L-BFGS-B:

In [109]:
nLL = lambda *args: - h.LogLikelihood(*args)

result = op.minimize( fun = nLL,
                     x0 = h._initRandomValues(),
                     args = (t,j,x),
                     method = 'L-BFGS-B',
                     jac = False,
                     bounds = h._paramBounds(),
                     options = {'disp': True} )

Investigating optimization result:

In [115]:
arr = result["x"]

print("""eta:
        {0} - estimated
        {1} - true\n""".format(arr[:2],np.asarray(eta)))
print("""Q:
        {0} - estimated
        {1} - true\n""".format(arr[2:6],np.asarray(Q)))
print("""alpha:
        {0} - estimated
        {1} - true\n""".format(arr[6:8],np.asarray(alpha)))
print("""alph:
        {0} - estimated
        {1} - true\n""".format(arr[8:10],np.asarray(alph)))
print("""beta:
        {0} - estimated
        {1} - true\n""".format(arr[10:12],np.asarray(beta)))
print("""gmma:
        {0} - estimated
        {1} - true\n""".format(arr[12:14],np.asarray(gmma)))

print("""Spr(Q) of estimated model: {0}""".format(h.ibParam.getSpectralRadius()))

eta:
        [ 0.033  0.039] - estimated
        [ 0.021  0.029] - true

Q:
        [ 0.544  0.103  0.586  0.   ] - estimated
        [ 0.61  0.16  0.6   0.06] - true

alpha:
        [ 0.006  0.007] - estimated
        [ 0.005  0.005] - true

alph:
        [ 6.074  2.87 ] - estimated
        [ 0.47  1.1 ] - true

beta:
        [  0.     14.941] - estimated
        [ 0.22  0.  ] - true

gmma:
        [ 3.336  0.   ] - estimated
        [ 0.  0.] - true

Spr(Q) of estimated model: 0.638632497252
